# Seldon V2 Kubernetes Examples

 * Create a kubernetes cluster with local auth to it
 * Install Kafka - see `kafka/strimzi` folder
 * Build if needed and place `seldon` binary in your path
 * Install Seldon on Kubernetes
   * Run `make deploy-k8s` from top level folder


In [ ]:
MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP
MESH_IP

In [ ]:
!which seldon

## Model

In [ ]:
!cat ./models/sklearn-iris-gs.yaml

In [ ]:
!kubectl create -f ./models/sklearn-iris-gs.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

In [ ]:
!kubectl get model iris -n seldon-mesh -o jsonpath='{.status}' | jq .

In [ ]:
!seldon model infer --model-name iris --inference-host ${MESH_IP} --inference-port 80 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

In [ ]:
!seldon model infer --model-name iris --inference-mode grpc --inference-host ${MESH_IP} --inference-port 80 \
   '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' | jq .

In [ ]:
!kubectl get server mlserver -n seldon-mesh -o jsonpath='{.status}' | jq .

In [ ]:
!kubectl delete -f ./models/sklearn-iris-gs.yaml

## Experiment

In [ ]:
!cat ./experiments/sklearn1.yaml

In [ ]:
!cat ./experiments/sklearn2.yaml 

In [ ]:
!kubectl create -f ./experiments/sklearn1.yaml
!kubectl create -f ./experiments/sklearn2.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

In [ ]:
!cat ./experiments/ab-default-model.yaml 

In [ ]:
!kubectl create -f ./experiments/ab-default-model.yaml 

In [ ]:
!kubectl wait --for condition=ready --timeout=300s experiment --all -n seldon-mesh

In [ ]:
!seldon model infer --inference-host ${MESH_IP} --inference-port 80 -i 50 --model-name iris \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

In [ ]:
!kubectl delete -f ./experiments/ab-default-model.yaml 
!kubectl delete -f ./experiments/sklearn1.yaml
!kubectl delete -f ./experiments/sklearn2.yaml

## Pipeline - smoke test - single model

In [ ]:
!cat ./models/sklearn-iris-gs.yaml

In [ ]:
!kubectl create -f ./models/sklearn-iris-gs.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

In [ ]:
!cat ./pipelines/iris.yaml

In [ ]:
!kubectl create -f ./pipelines/iris.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s pipeline --all -n seldon-mesh

In [ ]:
!seldon pipeline infer -p iris --inference-host ${MESH_IP} --inference-port 80 \
   '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

In [ ]:
!kubectl delete -f ./pipelines/iris.yaml

In [ ]:
!kubectl delete -f ./models/sklearn-iris-gs.yaml

## Pipeline - model chain

In [ ]:
!cat ./models/tfsimple1.yaml
!cat ./models/tfsimple2.yaml

In [ ]:
!kubectl create -f ./models/tfsimple1.yaml
!kubectl create -f ./models/tfsimple2.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

In [ ]:
!cat ./pipelines/tfsimples.yaml

In [ ]:
!kubectl create -f ./pipelines/tfsimples.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s pipeline --all -n seldon-mesh

In [ ]:
!seldon pipeline infer -p tfsimples --inference-mode grpc --inference-host ${MESH_IP} --inference-port 80 \
    '{"model_name":"simple","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]}]}' | jq .

In [ ]:
!kubectl delete -f ./pipelines/tfsimples.yaml

In [ ]:
!kubectl delete -f ./models/tfsimple1.yaml
!kubectl delete -f ./models/tfsimple2.yaml

## Pipeline - model join

In [ ]:
!cat ./models/tfsimple1.yaml
!cat ./models/tfsimple2.yaml
!cat ./models/tfsimple3.yaml

In [ ]:
!kubectl create -f ./models/tfsimple1.yaml
!kubectl create -f ./models/tfsimple2.yaml
!kubectl create -f ./models/tfsimple3.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

In [ ]:
!cat ./pipelines/tfsimples-join.yaml

In [ ]:
!kubectl create -f ./pipelines/tfsimples-join.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s pipeline --all -n seldon-mesh

In [ ]:
!seldon pipeline infer -p join --inference-mode grpc --inference-host ${MESH_IP} --inference-port 80 \
    '{"model_name":"simple","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]}]}' | jq .

In [ ]:
!kubectl delete -f ./pipelines/tfsimples-join.yaml

In [ ]:
!kubectl delete -f ./models/tfsimple1.yaml
!kubectl delete -f ./models/tfsimple2.yaml
!kubectl delete -f ./models/tfsimple3.yaml